In [ ]:
import csv
import re
import time
from urllib.parse import urljoin, urlparse, urlunparse
import requests
from bs4 import BeautifulSoup

In [ ]:
BASE_LIST_URL = "https://mapadotacji.gov.pl/projekty/?search-program=29425&page_no={page}" # 29425 to id krajowego programu odbudowy
BASE_SITE = "https://mapadotacji.gov.pl"
OUTPUT_CSV = "kpo_project_links.csv"

PROJECT_URL_RE = re.compile(r"^https?://mapadotacji\.gov\.pl/projekty/(\d+)/?$", re.IGNORECASE)

MAX_EMPTY_PAGES = 3 # jezeli beda 3 puste strony z rzedu, program konczy dzialanie
MAX_PAGES = 9999

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; KPO-scraper/0.2;)"
}

In [ ]:
def normalize_absolute(href: str) -> str:
    """Normalizuje url"""
    abs_url = urljoin(BASE_SITE, href.strip())
    p = urlparse(abs_url)

    cleaned = urlunparse((p.scheme, p.netloc, p.path, "", "", ""))

    if not cleaned.endswith("/"):
        cleaned = cleaned + "/"
    return cleaned

In [31]:
def extract_project_links(html: str) -> list[tuple[str, str]]:
    soup = BeautifulSoup(html, "html.parser")
    results: list[tuple[str, str]] = []
    for a in soup.find_all("a", href=True):
        normalized = normalize_absolute(a["href"])
        m = PROJECT_URL_RE.match(normalized)
        if m:
            project_id = m.group(1)
            results.append((project_id, normalized))
    return results

In [32]:
def main():
    session = requests.Session()
    session.headers.update(HEADERS)

    seen_ids: set[str] = set()
    empty_streak = 0

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["project_id", "url"])

        for page in range(1, MAX_PAGES + 1):
            url = BASE_LIST_URL.format(page=page)
            try:
                resp = session.get(url, timeout=20)
            except requests.RequestException as e:
                print(f"Upsi, błąd na stronie {page}: {e}")
                break

            if resp.status_code != 200:
                print(f"Strona {page}: HTTP {resp.status_code} – kończę.")
                break

            links = extract_project_links(resp.text)
            new_count = 0
            for project_id, absolute_url in links:
                if project_id not in seen_ids:
                    seen_ids.add(project_id)
                    writer.writerow([project_id, absolute_url])
                    new_count += 1

            print(f"Strona {page}: znaleziono {len(links)} linków, nowych {new_count} (razem {len(seen_ids)}).")

            if new_count == 0:
                empty_streak += 1
                if empty_streak >= MAX_EMPTY_PAGES:
                    print(f"Brak nowych linków przez {MAX_EMPTY_PAGES} strony – kończę na stronie {page}.")
                    break
            else:
                empty_streak = 0

            time.sleep(0.6)

    print(f"Gotowe! Zapisano {len(seen_ids)} unikalnych linków do pliku \"{OUTPUT_CSV}\".")

if __name__ == "__main__":
    main()

Strona 1: znaleziono 30 linków, nowych 10 (razem 10).
Strona 2: znaleziono 30 linków, nowych 10 (razem 20).
Strona 3: znaleziono 30 linków, nowych 10 (razem 30).
Strona 4: znaleziono 30 linków, nowych 10 (razem 40).
Strona 5: znaleziono 30 linków, nowych 10 (razem 50).
Strona 6: znaleziono 30 linków, nowych 10 (razem 60).
Strona 7: znaleziono 30 linków, nowych 10 (razem 70).
Strona 8: znaleziono 30 linków, nowych 10 (razem 80).
Strona 9: znaleziono 30 linków, nowych 10 (razem 90).
Strona 10: znaleziono 30 linków, nowych 10 (razem 100).
Strona 11: znaleziono 30 linków, nowych 10 (razem 110).
Strona 12: znaleziono 30 linków, nowych 10 (razem 120).
Strona 13: znaleziono 30 linków, nowych 10 (razem 130).
Strona 14: znaleziono 30 linków, nowych 10 (razem 140).
Strona 15: znaleziono 30 linków, nowych 10 (razem 150).
Strona 16: znaleziono 30 linków, nowych 10 (razem 160).
Strona 17: znaleziono 30 linków, nowych 10 (razem 170).
Strona 18: znaleziono 30 linków, nowych 10 (razem 180).
Strona 19: